# DQN with `dm_control`

Here are some experiments using deep Q-Learning to solve simple continuous control tasks. I implemented [the original DQN](https://www.nature.com/articles/nature14236) and used it to solve several tasks in the [DeepMind Control Suite](https://arxiv.org/abs/1801.00690). The key innovations from DQN are:

1. Maintain a replay buffer of experiences from which minibatches are randomly drawn during training. This decreases correlations in the training data, thereby reducing variance in the updates.
2. Keep an additional Q network for calculating targets that is an 'outdated' version of the main Q network. Every `q_update_interval` updates the weights are copied from the main to the target q network. Updates are more stable because the target network is updated less frequently.

Q-Learning takes the max across actions, which is not ideal for continuous action spaces. In this implementation the action space is discretized, such that each action dimension can take a value in `linspace(action_min, actions_max, action_grid)`, where `action_grid=2` for this demo. The full action space is the cartesian product of the vectors for each dimension.

The demo is organized as follows:
1. **setup**
2. **solving tasks:** cartpole (balance+swingup), ball in cup, pendulum
3. **double DQN:** I implement [Double Q-Learning](https://arxiv.org/abs/1509.06461) and test whether it increases the accuracy of action-value estimates.
4. **encouraging exploration:** To increase exploration I used a simple trick to encourage optimism in the face of uncertainty. Namely, I pretrained the network to output optimistic action-values across the state-space, which encourages exploration in the early phases of learning.

# setup


 The heart of the algorithm can be found in:
- [`train_utils.train`](https://github.com/richard-warren/rl_sandbox/blob/e56c44d74ddd47cbd6c2dc37753ba95896f9b81d/dm_control_tests/train_utils.py#L87), which trains an [`Agent`](https://github.com/richard-warren/rl_sandbox/blob/e56c44d74ddd47cbd6c2dc37753ba95896f9b81d/dm_control_tests/agents.py#L18) given an agent and a `dm_control` environment. 
- [`Agent.update`](https://github.com/richard-warren/rl_sandbox/blob/e56c44d74ddd47cbd6c2dc37753ba95896f9b81d/dm_control_tests/agents.py#L67), which selects minibatches and performs network updates.

To increase training speed I found it helpful to:
- *Train on the CPU rather than GPU.* The Q network is very small. My CPU was faster than the GPU unless batch sizes were really large.
- *Perform forward passes on Numpy*. Network forward passes ended up being much faster with numpy than Tensorflow (again, unless batch sizes were really large).
- *Train multiple agents in parallel*. Training results could be somewhat idiosyncratic even with the same hyperparameters, so I train 12 agents in parallel to make sure the results are robust.

Below are some utility functions for plotting performance and showing rollouts for trained agents.


In [ ]:
%load_ext autoreload
%autoreload 2

from dm_control_tests import train_utils, plot_utils
from dm_control_tests.agents import Agent
import matplotlib.pyplot as plt
from dm_control import suite
from tqdm.auto import tqdm
import tensorflow as tf
import seaborn as sns
import numpy as np
import pickle
import os


tf.get_logger().setLevel('ERROR')
train_utils.disable_gpu()

# plot performance over training, averaged across agents
def plot_performance(x, data):
    ax = plt.axes(xlabel='episode', ylabel='return',
                  xlim=(x[0],x[-1]), ylim=(0,1000))
    data = np.array(data)
    if data.ndim==2:
        mean = data.mean(0)
        std = data.std(0)
        ax.plot(x, data.T, color=(0,0,0), alpha=.15)
        ax.plot(x, mean)
        ax.fill_between(x, mean+std, mean-std, alpha=.15)
    else:
        ax.plot(x, data)

# show a rollout for the agent with the best performance at the end of training
def show_best_agent_rollout(agents_dir, framerate=30, epsilon=.05):
    with open(os.path.join(agents_dir, 'training_data'), 'rb') as file:
        training_data = pickle.load(file)
    best_agent = np.argmax(training_data['avg_returns'][:,-1])
    agent, metadata = train_utils.load_agent(
        os.path.join(agents_dir, 'agent{:03d}'.format(best_agent)))
    env = suite.load(*metadata['domain_and_task'])
    return plot_utils.show_rollout_jupyter(agent, env, epsilon=epsilon, framerate=framerate)


### training
Each action dimension is discretized into `action_grid=2` actions. This makes training fast, but it also means a given action can't be set to `0`. The replay buffer has length `buffer_length=50000` and is initialized using random actions prior to training. The Q network has hidden layers of size `units_per_layer=(12,24)`.  The target Q network is updated after every `q_update_interval=100` updates of the main Q network.

The temporal resolution of `dm_control` is rather high. During exploration, especially with sparse rewards, the sequence of correctly chosen actions necessary to discover the reward may be quite long. I therefore repeat each action `action_repeats=2` times. Furthermore, I only perform network updates every `steps_per_update=4` iterations. The updates, rather than the physics simulations, were the speed bottleneck. Less frequent updates allow the agent to see more episodes with the same compute time.

I train for `episodes=200` episodes with `batch_size=64`, acting $\epsilon$-greedy with $\epsilon$ linearly annealed from `epsilon_start=1` to `epsilon_final=.1` over `epsilon_final_episode=100` episodes.

### evaluation
Finally, every `eval_interval=10` episodes I evaluate performance by averaging the return from 5 rollouts with $\epsilon = .05$. `nagents=12` agents are trained in parallel, and I plot the average += standard deviation of the return across these agents (with individual agents plotted as thin gray lines).

In [ ]:
# agent
agent_args = dict(
    action_grid = 2,            # number of discrete actions per action dimension
    units_per_layer = (12,24),  # hidden units per layer
    buffer_length = 50000,
    q_update_interval = 100,    # q updates per q_target update
    learning_rate = .001,       # learning rate (adam optimizer)       
)

# training
train_args = dict(
    episodes = 200,
    eval_interval = 10,
    batch_size = 64,
    action_repeats = 2,         # repeat each action this number of times during training
    steps_per_update = 4,       # environment steps before updating q
    gamma = .99,
    epsilon_start = 1,
    epsilon_final = .1,
    epsilon_final_episode = 100,   # episode at which epsilon_final is reached
)

# solving simple tasks

I worked on `dm_control` tasks with action space dimensionality <=2: `cartpole`, `ball_in_cup`, and `pendulum`. Reasonable policies seemed to emerge, although there was some variability across agents.

## cartpole (balance)

I use `train_utils.train_agents_parallel` to train multiple agents in parallel and save them to disk. Cartpole (balance) was solved fairly quickly:

In [ ]:
episode_num, avg_returns = train_utils.train_agents_parallel(
    ('cartpole', 'balance'), agent_args, train_args, n_agents=12, n_workers=12,
    save_dir='dm_control_tests/agents/cartpole_balance')
plot_performance(episode_num, avg_returns)

Here's an $\epsilon=.05$ rollout for the agent with the best performance at the end of training. It doesn't look like anything is happening, which is just what we want for this task:

In [ ]:
show_best_agent_rollout('dm_control_tests/agents/cartpole_balance', epsilon=.05)

## cartpole (swingup)

Swingup (non-sparse) is a bit more challenging, but DQN works here as well:

In [ ]:
episode_num, avg_returns = train_utils.train_agents_parallel(
    ('cartpole', 'swingup'), agent_args, train_args, n_agents=12, n_workers=12,
    save_dir='dm_control_tests/agents/cartpole_swingup')
plot_performance(episode_num, avg_returns)

In [ ]:
show_best_agent_rollout('dm_control_tests/agents/cartpole_swingup', epsilon=.05)

## ball in cup

Ball in cup has two action dimensions. Performance across agents was more variable here, but performance was good overall:

In [ ]:
episode_num, avg_returns = train_utils.train_agents_parallel(
    ('ball_in_cup', 'catch'), agent_args, train_args, n_agents=12, n_workers=12,
    save_dir='dm_control_tests/agents/ball_in_cup')
plot_performance(episode_num, avg_returns)

In [ ]:
show_best_agent_rollout('dm_control_tests/agents/ball_in_cup', epsilon=0)

## pendulum

To show how the action-value function evolves over time I train a single agent on `pendulum`. The state space is two dimensional, and with discretization $\mathcal{A}=(-1,1)$. This means we can make nice heat maps that show action preference over the two-dimensional state space.

First let's train the agent. I found it necessary to set $\gamma=1$ (the reward is sparse here, and the agent has to swing back and forth several times to get it, so too much discounting can be problematic).

In the `pendulum` environment the angle $\theta$ is expressed with two values, $(\cos(\theta), \sin(\theta))$ (I'm guessing this is to get rid of the discontinuity in an angular representation?). I convert to $\theta$ below so I can fit things into a nice two-dimensional plot.

In [ ]:
env = suite.load('pendulum', 'swingup')
train_utils.rand_seed_reset(env, 0)
agent = Agent(env.observation_spec(), env.action_spec(), **agent_args)
train_utils.initialize_buffer(agent, env, verbose=False)

# adjust training settings
train_args_temp = train_args.copy()
train_args_temp['gamma'] = 1

# create state space grid points
bins = 40
angle_lims = (-np.pi/2, np.pi/2)
vel_lims = (-10,10)
axis_grids = [np.linspace(lims[0], lims[1], num=bins) for lims in (angle_lims, vel_lims)]
grid = np.array(np.meshgrid(*axis_grids)).reshape(2, -1).T
grid_predict = np.vstack((np.cos(grid[:,0]), np.sin(grid[:,0]), grid[:,1])).T  # first two columns are cos(x), sin(x) instead of x

# this callback function will be evaluated over the course of training
def compute_prediction_grid(agent, env):
    q_map = agent.q.predict(grid_predict).reshape(bins, bins, 2)
    return q_map

episode_num, returns, q_maps = train_utils.train(
    agent, env, **train_args_temp, verbose=False, callback=compute_prediction_grid)
returns = [np.mean(r) for r in returns]
plot_performance(episode_num, returns);


The video below shows how preference for one action (blue) over the other (red) evolves over time. There is little preference initially (white), but a pretty structure emerges over training:

In [ ]:
preference_maps = [m[:,:,0] - m[:,:,1] for m in q_maps]
abs_max = np.abs(np.array(preference_maps)).max()
plot_utils.display_video(
    preference_maps, framerate=2, is_plot=True,
    imshow_args={'extent': angle_lims+vel_lims, 'cmap': 'bwr',
                 'vmin': -abs_max, 'vmax': abs_max},
    xlabel='angle', ylabel='velocity')

The value function is both pretty and functional. See a sample rollout here:

In [ ]:
plot_utils.show_rollout_jupyter(agent, env, epsilon=0, rand_seed=5)

However, whever the starting state (which is randomized across episodes) is very low, the agent can't figure it out. In attemps to train for long periods of time (data not shown) I also found that performance can suffer after an intial period of learning.

In [ ]:
plot_utils.show_rollout_jupyter(agent, env, epsilon=0, rand_seed=4)

# double q learning

The max operation in Q-Learning can lead to overly optimistic value functions. In Double-Q Learning two Q functions are learned in parallel (typically on independent samples of data). One network is used to evaluate the actions selected by the other network. If the functions are $Q^a$ and $Q^b$, then the target used when updated $Q^a$ is:

$$
\text{target}^a = R_{t+1} + \gamma Q^b(S_{t+1}, \text{argmax}_{a} Q^a(S_{t+1}, a) )
$$

[Double DQN](https://arxiv.org/abs/1509.06461) doesn't train networks on independent samples. Rather, it takes advantage of there already be two Q networks, which are (usually) different from one another. The target Q network, which should be more stable, is used for evaluating the actions selected by the main Q network.

As in the original paper, I would like to check that using doubld DQN increases the accuracy of the action-value function. For DQN and double DQN agents I'll compute actual returns from a series of rollouts and compare these to returns estimated by their respective value-functions.

In [ ]:
agent_args_dq = agent_args.copy()
agent_args_dq['double_dqn'] = True

episode_num, avg_returns = train_utils.train_agents_parallel(
    ('ball_in_cup', 'catch'), agent_args_dq, train_args, n_agents=12, n_workers=12,
    save_dir='dm_control_tests/agents/ball_in_cup_dq')
plot_performance(episode_num, avg_returns)

In [ ]:
show_best_agent_rollout('dm_control_tests/agents/ball_in_cup_dq', epsilon=0)

Now we'll assess evaluation accuracy for DQN and Double DQN by comparing actual returns with returns predicted by the Q function. For cartpole (balance) I'll collect returns from 10 agents with 5 rollouts each (for both DQN and Double DQN agents) and plot actual vs. predicted return densities.

One detail: Episodes being truncated after 1000 steps will cause later returns to be underestimated. Therefore, when computing returns the final reward $R_T$ is given the remaining weight in the gamma distribution, e.g.

\begin{align}
G_t &= \sum_{i=t}^{T-1} \gamma^{i-t} R_i + \sum_{i=T}^\infty \gamma^{i-t} R_T \\
&= \sum_{i=t}^{T-1} \gamma^{i-t} R_i + \gamma^{T-t} \sum_{i=0}^\infty \gamma^{i} R_{T} \\ 
&= \sum_{i=t}^{T-1} \gamma^{i-t} R_i + \frac{\gamma^{T-t} R_{T}}{1-\gamma}
\end{align}

I compute returns "backwards" using $G_t = R_t + \gamma G_{t+1}$. This means the above equation can be implemented by settings $G_T = \frac{R_T}{1-\gamma}$ and then computing recursively thereafter. I still want to avoid returns that start close to the end of an episode, so I only included returns for which $\gamma^{T-t}<.01$ (so the truncated portion of the $\gamma$ distribution is negligible).

In [ ]:
# compare DQN and Double DQN evaluation accuracy
n_agents = 10
n_rollouts = 5  # per agent
shake_things_up = False
gamma_thresh = .01
env = suite.load('cartpole', 'swingup')
agent_q  = 'dm_control_tests/agents/cartpole_swingup/'
agent_dq = 'dm_control_tests/agents/cartpole_swingup_dq/'


# returns will be truncated near end of episodes, so only consider early returns
# with enough samples s.t. discounting makes further samples negligible
last_ind = int(env._step_limit - np.argmax(
    np.power(train_args['gamma'], np.arange(env._step_limit)) < gamma_thresh))

q_data = {'returns': [], 'predictions': []}
dq_data = {'returns': [], 'predictions': []}

for agent_name, data in zip((agent_q, agent_dq), (q_data, dq_data)):  # for dqn, double dqn
    for agent_num in tqdm(range(n_agents)):
        agent = train_utils.load_agent(os.path.join(agent_name, 'agent{:03d}'.format(agent_num)))[0]
        for i in range(n_rollouts):
            predictions, rewards = [], []
            time_step = env.reset()
            if shake_things_up:
                env.physics.data.qpos[:] = np.random.randn(env.physics.model.nq)

            # perform rollout and get return predictions
            while not time_step.last():
                action = agent.select_action(time_step, train_args['epsilon_final'])
                action_idx = agent.index_from_action(action)
                time_step = env.step(action)
                rewards.append(time_step.reward)
                x = agent.get_observation_vector(time_step)
                predictions.append(agent.predict(x, agent.q).max())

            # compute actual returns in reverse
            returns = [rewards[-1] * (1/(1-train_args['gamma']))]  # multiply final return by integral of gamma function to account for episode termination
            for r in reversed(rewards[:-1]):
                returns.insert(0, r + train_args['gamma']*returns[0])  # G_t = G_t + \gamma G_{t+1}

            # store
            data['returns'] += returns[:last_ind]
            data['predictions'] += predictions[:last_ind]




Here are densities for actual vs. predicted return. Both DQN and Double DQN were fairly, and comparably, accurate. I suspect the benefits of Double DQN are minimal because I copy weights to the Q target network every `100` updates, which is quite frequent compared to the `10000` used in the original paper. 

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15,5))

for i, data, title in zip(range(2), (q_data, dq_data), ('DQN', 'Double DQN')):
#     ax[i].scatter(data['returns'], data['predictions'], alpha=.1)
    ax[i].set(xlim=(0,100), ylim=(0,100), xlabel='actual return', ylabel='predicted return', title=title)
    sns.kdeplot(data['returns'], data['predictions'], ax=ax[i], shade=True, shade_lowest=False, cmap='Reds')
    ax[i].plot([0,100], [0,100], color=(.2,.2,.2))
    

# optimistic initializations
for sparse reward agent needs to accidentally discover correct action sequence // can be very hard when that sequence is long // initializing optimisitic values of q can hopefully encourage initial exploration and speed up learning...

In [ ]:
# todo: determine if optimism helps...

# sandbox...

In [ ]:
# train agent in parallel

agent_args_temp = agent_args.copy()
train_args_temp = train_args.copy()

agent_args_temp['double_dqn'] = True
train_args_temp['gamma'] = 1


episode_num, avg_returns = train_utils.train_agents_parallel(
    ('pendulum', 'swingup'), agent_args_temp, train_args_temp, n_agents=12, n_workers=12,
    save_dir='dm_control_tests/agents/pendulum_swingup_dq')
plot_performance(episode_num, avg_returns)

In [ ]:
# for reloading previous training data...
with open('dm_control_tests/agents/pendulum_swingup/training_data', 'rb') as f:
    training_data = pickle.load(f)
episode_num = training_data['episode_num']
avg_returns = training_data['avg_returns']
plot_performance(episode_num, avg_returns)
